In [1]:
import pandas as pd
from os import getcwd, listdir
from os.path import join, isfile, exists, abspath, pardir
from sys import path
import re

##### Configs

In [2]:
database = "sdm"
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")

In [3]:
path.append(scripts_dir)
from connect import Neo4jConnection

In [4]:
driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

##### Helper methods

In [5]:
def run_query(query):
    """
    Basic wrapper around 'driver' object
    """
    return driver.query(query)

In [6]:
def run_query_with_result(query):
    """
    Basic wrapper around 'driver' object
    """
    return driver.query_with_result(query)

In [7]:
q = """
        Match (d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper' and toInteger(d.cited_count) > 0
        With j.name as conference, d.title as paper, sum(toInteger(d.cited_count)) as cited_count
        order by cited_count desc
        return conference, collect(paper)[..3] as papers, collect(cited_count)[..3] as total_cited_count
    """

In [8]:
d, c = run_query_with_result(q)

type(result): <class 'neo4j.work.result.Result'>
list(result.keys()): ['conference', 'papers', 'total_cited_count']


In [13]:
d

,0,1,2
0,Proceedings of SPIE - The International Societ...,[Near-to-eye electroholography via guided-wave...,"[165, 45, 30]"
1,Proceedings of the ASME Design Engineering Tec...,[Kinematics and discretization of curved-fold ...,"[65, 52, 39]"
2,Proceedings of the ASME Turbo Expo,[Predicting efficiency of a turbine driven by ...,"[54, 24, 18]"
3,Proceedings of the American Control Conference,[A Class of Logistic Functions for Approximati...,"[24, 12, 8]"
4,Optics InfoBase Conference Papers,[3D hydrodynamic focusing for optofluidics usi...,"[18, 9]"
...,...,...,...
89,Proceedings of the 20th International Conferen...,[Using instagram to increase student engagemen...,"[1, 1]"
90,"2018 Aviation Technology, Integration, and Ope...","[Exploration of three dimensional, hierarchica...",[1]
91,Proceedings of the International Masonry Socie...,[Strut-and-tie models for masonry walls with o...,[1]
92,"Wind Energy Symposium, 2018",[Benefits of two turbine rotor diameters and h...,[1]


In [8]:
from neo4j import GraphDatabase

In [9]:
driver = GraphDatabase.driver("bolt://localhost:7687")

In [59]:
def get_two_tx(tx):
    result = tx.run("""
        Match (d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper' and toInteger(d.cited_count) > 0
        With j.name as conference, d.title as paper, sum(toInteger(d.cited_count)) as cited_count
        order by cited_count desc
        return conference, collect(paper)[..3] as papers, collect(cited_count)[..3] as total_cited_count
    """)
    values = list()
    for ix, record in enumerate(result):
        # if ix > 1:
        #     break
        value = record.values()
        if value:
            # if isinstance(value, list) and len(value):
            #     value = value[0]
            values.append(value)
    info = result.consume()  # discard the remaining records if there are any
    # use the info for logging etc.
    return values, info

In [60]:
with driver.session(database=database) as session:
    values, info = session.read_transaction(get_two_tx)

In [ ]:
pd.DataFrame(values)

In [85]:
values = list()
for ix, record in enumerate(run_query(q)):
    # if ix > 1:
    #     break
    value = record.values()
    if value:
        # if isinstance(value, list) and len(value):
        #     value = value[0]
        values.append(value)
# info = r.consume()

In [86]:
values

[]